# Bokeh服务器自动刷新应用

通过`bokeh serve app.py`可以启动Bokeh服务器并运行`app.py`应用程序。每次用浏览器访问`http://localhost:5006/app`都会在服务器中创建一个新的`Session`对象，并在其中执行`app.py`中的代码。但是代码并不会重新从文件读入，因此在调试`app.py`代码时需要不断地重新运行Bokeh服务器，为调试工作带来不必要的麻烦。

在`app.py`中载入下面的`bokeh_reload`模块，会对Bokeh中的`CodeRunner`打上自动重新读入代码的补丁。

In [2]:
%%file bokeh_reload.py
from os import path
from bokeh.application.handlers import code_runner

def run(self, module, post_check):
    import ast

    if path.exists(self._path):
        with open(self._path, 'r') as f:
            source = f.read()
        nodes = ast.parse(source, self._path)
        self._code = compile(nodes, filename=self._path, mode='exec', dont_inherit=True)
        self._source = source

    self._old_run(module, post_check)

if not hasattr(code_runner.CodeRunner, "_old_run"):
    code_runner.CodeRunner._old_run = code_runner.CodeRunner.run
    code_runner.CodeRunner.run = run

Overwriting bokeh_reload.py


下面是测试用的代码：

In [17]:
%%file bokeh_test_reload.py
from bokeh.plotting import curdoc
from bokeh.models import Div, Button
from bokeh.layouts import column
import bokeh_reload

div = Div()
button = Button()

def callback():
    div.text = "hello world"

button.on_click(callback)
curdoc().add_root(column(div, button))

Overwriting bokeh_test_reload.py


运行下面的代码启动服务器，并在浏览器中打开`http://localhost:5006/bokeh_test_reload`。

In [18]:
import subprocess
process = subprocess.Popen(['start', 'bokeh', 'serve', 'bokeh_test_reload.py'], shell=True)

修改上面的代码，例如为按钮设置标签文本`Button(label="Click Me")`，然后刷新浏览器，可以看到程序自动被重新载入了。

In [19]:
%%file bokeh_test_reload.py
from bokeh.plotting import curdoc
from bokeh.models import Div, Button
from bokeh.layouts import column
import bokeh_reload

div = Div()
button = Button(label="Click Me")

def callback():
    div.text = "hello world"

button.on_click(callback)
curdoc().add_root(column(div, button))

Overwriting bokeh_test_reload.py
